In [1]:
colab = False

if colab:
    from google.colab import drive
    drive.mount('/content/gdrive')
    !git clone https://github.com/deepmind/pycolab.git
    !git clone https://github.com/nicoladainese96/RelationalModule.git
    !pip install pycolab

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from RelationalModule import ActorCritic
from RelationalModule import train_agent as train
from importlib import reload

In [3]:
reload(train)

<module 'RelationalModule.train_agent' from '/home/nicola/Nicola_unipd/MasterThesis/RelationalDeepRL/RelationalModule/train_agent.py'>

In [4]:
game_params = dict(grid_size=12,
                solution_length=[1],
                num_forward = [1], # number of distractors
                num_backward=[0], # just set to 0 for now
                branch_length=1 # length of forward distractors
               )

In [5]:

HPs = dict(action_space=4,
           lr=1e-2,
           gamma=0.99,
           TD=True,
           twin=True,
           n_steps = 3
           )
if colab:
    HPs['device'] = 'cuda'
else:
    HPs['device'] = 'cpu'

print('device: ', HPs['device'])  
agent = ActorCritic.BoxWorldA2C(**HPs)

device:  cpu
========== A2C HyperParameters ==========
Discount factor:  0.99
Learning rate:  0.01
Action space:  4
Temporal Difference learning:  True
Twin networks:  True
Update critic target factor:  1.0
n_steps for TD:  3
Device used:  cpu


========== A2C Architecture ==========
Actor architecture: 
 BoxWorldActor(
  (boxnet): BoxWorldNet(
    (net): Sequential(
      (0): ExtractEntities(
        (embed): Embedding(117, 3)
        (net): Sequential(
          (0): Conv2d(3, 12, kernel_size=(2, 2), stride=(1, 1))
          (1): ReLU()
          (2): Conv2d(12, 24, kernel_size=(2, 2), stride=(1, 1))
          (3): ReLU()
        )
      )
      (1): RelationalModule(
        (net): Sequential(
          (0): PositionalEncoding(
            (projection): Linear(in_features=26, out_features=256, bias=True)
          )
          (1): AttentionBlock(
            (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [6]:
%%time
results = train.train_boxworld(agent, game_params, n_episodes = 10, return_agent=True)
score, asymptotic_score, asymptotic_std, trained_agent, time_profile = results

/home/nicola/anaconda3/envs/torch/lib/python3.7/site-packages/pycolab/ascii_art.py:318: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  art = np.vstack(np.fromstring(line, dtype=np.uint8) for line in art)


Time playing the episode: 1.65 s
Episode 1 - reward: -1
n_step_rewards.shape:  (89,)
rewards.shape:  (89,)
n_step_rewards:  [ 0.      0.      0.      0.      0.      0.      0.      0.      0.
  0.      0.      0.      0.      0.      0.      0.      0.      0.
  0.      0.      0.      0.      0.      0.      0.      0.      0.
  0.      0.      0.      0.      0.      0.      0.      0.      0.
  0.      0.      0.      0.      0.      0.      0.      0.      0.
  0.      0.      0.      0.      0.      0.      0.      0.      0.
  0.      0.      0.      0.      0.      0.      0.      0.      0.
  0.      0.      0.      0.      0.      0.      0.      0.      0.
  0.      0.      0.      0.      0.      0.      0.      0.      0.
  0.      0.      0.      0.      0.     -0.9801 -0.99   -1.    ]
rewards:  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0. 

V_trg.shape (after critic):  torch.Size([121])
V_trg.shape (after sum):  torch.Size([121])
V_trg.shape (after squeeze):  torch.Size([121])


KeyboardInterrupt: 

On my PC:

 Average time for playing an episode: 1.97428691 <br>
 Average time for updating the agent: 26.46678982

 On Colab notebook:

 Average time for playing an episode:  1.11658633 <br>
 Average time for updating the agent: 18.63836179


With GPU

Average time for playing an episode: 0.43020332 <br>
Average time for updating the agent: 0.64481306


In [7]:
plt.figure(figsize=(8,6))
n_epochs = np.arange(len(score))
plt.plot(n_epochs, score, alpha=0.9)
plt.title("Performance", fontsize=16)
plt.xlabel("Number of epochs", fontsize=16)
plt.ylabel("Total reward (max=500)", fontsize=16)
plt.show()

NameError: name 'score' is not defined

<Figure size 576x432 with 0 Axes>